In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
import sys
import os


In [ ]:
sys.path.insert(0, os.path.abspath("../src/"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

In [ ]:
from evaluation.eval_utils.compute_scores import gen_evaluation, compute_numbers

In [ ]:
masks_path = Path("../src/evaluation/")
data_path = Path("../datasets/VOC2007/")
segmentations_path = Path("../datasets/VOC2007/VOCdevkit/VOC2007/SegmentationClass/")
dataset_name = "VOC"
model_name = "vgg16"
model_path = "../src/checkpoints/pretrained_classifiers/vgg16_voc.ckpt"
# method = "extremal_perturbation"
# method = "guided_backprop"

method = "grad_cam"
# method = "rise"
# method = "explainer"
# method = "dabkowski"


In [ ]:
mask, seg_mask, p, p_mask, p_background, category_id, x = next(gen_evaluation(data_path, masks_path, segmentations_path, dataset_name, model_name, model_path, method).__iter__())

In [ ]:
plt.figure(figsize=(10,3))
plt.subplot(131)
plt.imshow(np.transpose(x,[1,2,0]), vmin=0, vmax=1)
plt.subplot(132)
plt.imshow(seg_mask, cmap="gray", vmin=0, vmax=1)
plt.colorbar()
plt.subplot(133)
plt.imshow(mask, cmap="gray", vmin=0, vmax=1)
plt.colorbar()


# Compute results

In [ ]:
res = compute_numbers(data_path, masks_path, segmentations_path, dataset_name, model_name, model_path, method)
d_f1_25,d_f1_50,d_f1_75,c_f1,a_f1s, aucs, d_IOU, c_IOU, sal, over, background_c, mask_c, sr = res

In [ ]:
print("Discrete F1 25 : {:.3f}".format(np.mean(np.array(d_f1_25))))
print("Discrete F1 50 : {:.3f}".format(np.mean(np.array(d_f1_50))))
print("Discrete F1 75 : {:.3f}".format(np.mean(np.array(d_f1_75))))
print("Continuous F1  : {:.3f}".format(np.mean(np.array(c_f1))))


In [ ]:
print("Continuous IOU : {:.3f}".format(np.mean(np.array(c_IOU))))
print("Discrete   IOU : {:.3f}".format(np.mean(np.array(d_IOU))))


In [ ]:
print("Saliency           : {:.3f}".format(np.mean(np.array(sal))))


In [ ]:
print("Overlap            : {:.3f}".format(np.mean(np.array(over))))
print("Mask coverage      : {:.3f}".format(np.mean(np.array(mask_c))))
print("Background coverage: {:.3f}".format(np.mean(np.array(background_c))))
print("Sim ratio.         : {:.3f}".format(np.mean(np.array(sr))))

# Compute all metrics

In [ ]:
try:
    results = np.load("results.npz", allow_pickle=True)["results"].item()
except:
    results = {}

In [ ]:
masks_path = Path("../src/evaluation/")
data_base_path = Path("../datasets/")
VOC_segmentations_path = Path("../datasets/VOC2007/VOCdevkit/VOC2007/SegmentationClass/")
COCO_segmentations_path = Path("../evaluations/coco_segmentations/")

datasets = ["VOC", "COCO"]
classifiers = ["vgg16", "resnet50"]
vgg16_voc_checkpoint = "../src/checkpoints/pretrained_classifiers/vgg16_voc.ckpt"
vgg16_coco_checkpoint = "../src/checkpoints/pretrained_classifiers/vgg16_coco.ckpt"
resnet50_voc_checkpoint = "../src/checkpoints/pretrained_classifiers/resnet50_voc.ckpt"
resnet50_coco_checkpoint = "../src/checkpoints/pretrained_classifiers/resnet50_coco.ckpt"

methods = ["extremal_perturbations",  "grad_cam", "rise", "explainer", 
          "rt_saliency", "guided_backprop", "0.5", "0", "1", "perfect"]

In [ ]:
for dataset in datasets:
    if not(dataset in results):
        results[dataset] = {}
    for classifier in classifiers:
        if not(classifier in results[dataset]):
            results[dataset][classifier] = {}
        for method in methods:
            if not(method in results[dataset][classifier]):
                results[dataset][classifier][method] = {}            
            try:
                if dataset == "VOC":
                    data_path = data_base_path / "VOC2007"
                    segmentations_path = VOC_segmentations_path
                    if classifier == "vgg16":
                        model_path = vgg16_voc_checkpoint
                    elif classifier == "resnet50":
                        model_path = resnet50_voc_checkpoint
                elif dataset == "COCO":
                    data_path = data_base_path / "COCO2014"
                    segmentations_path = COCO_segmentations_path
                    if classifier == "vgg16":
                        model_path = vgg16_coco_checkpoint
                    elif classifier == "resnet50":
                        model_path = resnet50_coco_checkpoint

                d_f1_25,d_f1_50,d_f1_75,c_f1,a_f1s, aucs, d_IOU, c_IOU, sal, over, background_c, mask_c, sr = compute_numbers(data_path=data_path,
                                                                                                                              masks_path=masks_path, 
                                                                                                                              segmentations_path=segmentations_path, 
                                                                                                                              dataset_name=dataset, 
                                                                                                                              model_name=classifier, 
                                                                                                                              model_path=model_path, 
                                                                                                                              method=method)


                d = {}
                d["d_f1_25"] = d_f1_25
                d["d_f1_50"] = d_f1_50
                d["d_f1_75"] = d_f1_75
                d["d_f1"] = ((np.array(d_f1_25) + np.array(d_f1_50) + np.array(d_f1_75)) /3).tolist()
                d["c_f1"] = c_f1
                d["a_f1s"] = a_f1s
                d["aucs"] = aucs
                d["d_IOU"] = d_IOU
                d["c_IOU"] = c_IOU
                d["sal"] = sal
                d["over"] = over
                d["background_c"] = background_c
                d["mask_c"] = mask_c
                d["sr"] = sr
                results[dataset][classifier][method] = d
                print("Scores computed for: {} - {} - {}".format(dataset, classifier, method))
            except:
                print("Cannot compute scores for: {} - {} - {}!".format(dataset, classifier, method))

np.savez("results.npz", results=results)